In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#read data
quantile_calib_df = pd.read_csv('data\Financial Timeseries\quantile_calib.csv') 
print(quantile_calib_df.head())
quantile_test_df = pd.read_csv('data\Financial Timeseries\quantile_test.csv')
print(quantile_test_df.head())

pred_calib_df = pd.read_csv('data\Financial Timeseries\pred_calib.csv')
print(pred_calib_df.head())
pred_test_df = pd.read_csv('data\Financial Timeseries\pred_test.csv')
print(pred_test_df.head())

         Date Ticker  pred_lower_calib  pred_upper_calib   y_calib
0  2013-01-02   AAPL         -0.031219          0.032132  0.031682
1  2013-01-03   AAPL         -0.028245          0.028795 -0.012622
2  2013-01-04   AAPL         -0.028535          0.028392 -0.027855
3  2013-01-07   AAPL         -0.035386          0.032612 -0.005882
4  2013-01-08   AAPL         -0.026758          0.034949  0.002691
         Date Ticker  pred_lower_test  pred_upper_test    y_test
0  2020-01-02   AAPL        -0.017743         0.015796  0.022816
1  2020-01-03   AAPL        -0.014777         0.016668 -0.009722
2  2020-01-06   AAPL        -0.016409         0.013452  0.007968
3  2020-01-07   AAPL        -0.015384         0.017074 -0.004703
4  2020-01-08   AAPL        -0.014702         0.017137  0.016086
         Date Ticker  pred_calib   y_calib
0  2013-01-02   AAPL   -0.000876  0.031682
1  2013-01-03   AAPL   -0.000083 -0.012622
2  2013-01-04   AAPL    0.002104 -0.027855
3  2013-01-07   AAPL    0.001792 -0.

In [3]:
# coverage rate QR calibration
violation_lower_calib = quantile_calib_df['y_calib'] < quantile_calib_df['pred_lower_calib']
violation_upper_calib = quantile_calib_df['y_calib'] > quantile_calib_df['pred_upper_calib']
violation_calib = violation_lower_calib | violation_upper_calib
coverage_rate_calib = 1 - violation_calib.mean()
print(f'Coverage rate QR calibration: {coverage_rate_calib:.4f}')

# coverage rate QR test
violation_lower_test = quantile_test_df['y_test'] < quantile_test_df['pred_lower_test']
violation_upper_test = quantile_test_df['y_test'] > quantile_test_df['pred_upper_test']
violation_test = violation_lower_test | violation_upper_test
coverage_rate_test = 1 - violation_test.mean()
print(f'Coverage rate QR test: {coverage_rate_test:.4f}')

#give coverage for before conformalisation

Coverage rate QR calibration: 0.8875
Coverage rate QR test: 0.8632


In [4]:
# R^2 and MAE mean calibration
from sklearn.metrics import r2_score, mean_absolute_error
r2_calib = r2_score(pred_calib_df['y_calib'], pred_calib_df['pred_calib'])
mae_calib = mean_absolute_error(pred_calib_df['y_calib'], pred_calib_df['pred_calib'])
print(f'R^2 mean calibration: {r2_calib:.4f}')
print(f'MAE mean calibration: {mae_calib:.4f}')

# R^2 and MAE mean test
r2_test = r2_score(pred_test_df['y_test'], pred_test_df['pred_test'])
mae_test = mean_absolute_error(pred_test_df['y_test'], pred_test_df['pred_test'])
print(f'R^2 mean test: {r2_test:.4f}')
print(f'MAE mean test: {mae_test:.4f}')

R^2 mean calibration: -0.0051
MAE mean calibration: 0.0101
R^2 mean test: -0.0124
MAE mean test: 0.0143


In [5]:
from ConformalPredictor import ConformalPredictor

In [6]:
ticker_list = pred_test_df['Ticker'].unique()
print(f'Tickers: {ticker_list}')
print(f'Number of tickers: {len(ticker_list)}')

#get ticker with the most rows in pred_calib_df
ticker_most_rows = pred_calib_df['Ticker'].value_counts().idxmax()
print(f'Ticker with most rows in pred_calib_df: {ticker_most_rows}')
print(f'Number of rows in pred_calib_df for ticker {ticker_most_rows}: {len(pred_calib_df[pred_calib_df["Ticker"] == ticker_most_rows])}')

# get all tickers with 1762 rows in pred_calib_df
tickers_1762_rows = pred_calib_df['Ticker'].value_counts()[pred_calib_df['Ticker'].value_counts() == 1762].index.tolist()
ticker_list = tickers_1762_rows
print(f'Tickers with 1762 rows in pred_calib_df: {tickers_1762_rows}')
print(f'Number of tickers with 1762 rows in pred_calib_df: {len(tickers_1762_rows)}')
#print num tickers list
print(f'Number of tickers list: {len(ticker_list)}')


Tickers: ['AAPL' 'ABBV' 'ABT' 'ACN' 'ADBE' 'AIG' 'AMD' 'AMGN' 'AMT' 'AMZN' 'AVGO'
 'AXP' 'BA' 'BAC' 'BK' 'BKNG' 'BLK' 'BMY' 'BRK-B' 'C' 'CAT' 'CHTR' 'CL'
 'CMCSA' 'COF' 'COP' 'COST' 'CRM' 'CSCO' 'CVS' 'CVX' 'DE' 'DHR' 'DIS'
 'DUK' 'EMR' 'FDX' 'GD' 'GE' 'GILD' 'GM' 'GOOG' 'GOOGL' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'INTU' 'ISRG' 'JNJ' 'JPM' 'KO' 'LIN' 'LLY' 'LMT' 'LOW' 'MA'
 'MCD' 'MDLZ' 'MDT' 'MET' 'META' 'MMM' 'MO' 'MRK' 'MS' 'MSFT' 'NEE' 'NFLX'
 'NKE' 'NOW' 'NVDA' 'ORCL' 'PEP' 'PFE' 'PG' 'PLTR' 'PM' 'PYPL' 'QCOM'
 'RTX' 'SBUX' 'SCHW' 'SO' 'SPG' 'T' 'TGT' 'TMO' 'TMUS' 'TSLA' 'TXN' 'UNH'
 'UNP' 'UPS' 'USB' 'V' 'VZ' 'WFC' 'WMT' 'XOM']
Number of tickers: 101
Ticker with most rows in pred_calib_df: AAPL
Number of rows in pred_calib_df for ticker AAPL: 1762
Tickers with 1762 rows in pred_calib_df: ['AAPL', 'INTU', 'ORCL', 'NVDA', 'NKE', 'NFLX', 'NEE', 'MSFT', 'MS', 'MRK', 'MO', 'MMM', 'MET', 'MDT', 'MDLZ', 'MCD', 'MA', 'LOW', 'LMT', 'LLY', 'LIN', 'KO', 'JPM', 'PEP', 'PFE', 'PG', 'TSLA', 'WMT', '

In [7]:
CongformalPredictor = ConformalPredictor(alpha=0.1)  # Ok (except typo)
ticker_results_standard_CP = []
ticker_results_CQR = []
ticker_results_ACI = []
#split data by ticker
for ticker in ticker_list:
    # mean prediction
    calib_df = pred_calib_df[pred_calib_df['Ticker'] == ticker]
    calib_pred = calib_df['pred_calib']
    y_calib = calib_df['y_calib']
    test_df = pred_test_df[pred_test_df['Ticker'] == ticker]
    pred_test = test_df['pred_test']
    y_test = test_df['y_test']

    # quantile regression prediction
    calib_quantile = quantile_calib_df[quantile_calib_df['Ticker'] == ticker]
    y_calib_quantile = calib_quantile['y_calib']
    pred_lower_calib = calib_quantile['pred_lower_calib']
    pred_upper_calib = calib_quantile['pred_upper_calib']
    test_quantile = quantile_test_df[quantile_test_df['Ticker'] == ticker]
    y_test_quantile = test_quantile['y_test']
    pred_lower_test = test_quantile['pred_lower_test']
    pred_upper_test = test_quantile['pred_upper_test']

    # standard CP
    standard_CP = CongformalPredictor.standard_cp(calib_pred, y_calib, pred_test, y_test)
    ticker_results_standard_CP.append(standard_CP)

    # CQR
    CQR = CongformalPredictor.cqr(pred_lower_calib, pred_upper_calib, y_calib_quantile, pred_lower_test, pred_upper_test, y_test_quantile)
    ticker_results_CQR.append(CQR)

    # ACI
    ACI = CongformalPredictor.aci(calib_pred, y_calib, pred_test, y_test)
    ticker_results_ACI.append(ACI)


In [8]:
# Print Coverage for for each ticker standard CP
print('Coverage for each ticker standard CP:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_standard_CP[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_standard_CP[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_standard_CP[i]['upper_bound'] - ticker_results_standard_CP[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker standard CP:
Ticker: AAPL, Coverage: 0.8240
Avg. length: 0.0490
Ticker: INTU, Coverage: 0.7107
Avg. length: 0.0415
Ticker: ORCL, Coverage: 0.7802
Avg. length: 0.0388
Ticker: NVDA, Coverage: 0.7424
Avg. length: 0.0709
Ticker: NKE, Coverage: 0.7711
Avg. length: 0.0404
Ticker: NFLX, Coverage: 0.8980
Avg. length: 0.0790
Ticker: NEE, Coverage: 0.7356
Avg. length: 0.0320
Ticker: MSFT, Coverage: 0.8051
Avg. length: 0.0426
Ticker: MS, Coverage: 0.8444
Avg. length: 0.0524
Ticker: MRK, Coverage: 0.8557
Avg. length: 0.0367
Ticker: MO, Coverage: 0.8505
Avg. length: 0.0360
Ticker: MMM, Coverage: 0.7968
Avg. length: 0.0352
Ticker: MET, Coverage: 0.8248
Avg. length: 0.0469
Ticker: MDT, Coverage: 0.8082
Avg. length: 0.0360
Ticker: MDLZ, Coverage: 0.8875
Avg. length: 0.0383
Ticker: MCD, Coverage: 0.8240
Avg. length: 0.0294
Ticker: MA, Coverage: 0.8369
Avg. length: 0.0432
Ticker: LOW, Coverage: 0.8187
Avg. length: 0.0424
Ticker: LMT, Coverage: 0.8338
Avg. length: 0.0338
Ticker: 

In [9]:
# Print Coverage for for each ticker CQR
print('Coverage for each ticker CQR:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_CQR[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_CQR[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_CQR[i]['upper_bound'] - ticker_results_CQR[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker CQR:
Ticker: AAPL, Coverage: 0.8814
Avg. length: 0.0564
Ticker: INTU, Coverage: 0.8580
Avg. length: 0.0618
Ticker: ORCL, Coverage: 0.8520
Avg. length: 0.0491
Ticker: NVDA, Coverage: 0.8693
Avg. length: 0.0944
Ticker: NKE, Coverage: 0.8678
Avg. length: 0.0554
Ticker: NFLX, Coverage: 0.9177
Avg. length: 0.0797
Ticker: NEE, Coverage: 0.8708
Avg. length: 0.0498
Ticker: MSFT, Coverage: 0.8784
Avg. length: 0.0527
Ticker: MS, Coverage: 0.8867
Avg. length: 0.0577
Ticker: MRK, Coverage: 0.8761
Avg. length: 0.0409
Ticker: MO, Coverage: 0.8920
Avg. length: 0.0411
Ticker: MMM, Coverage: 0.8625
Avg. length: 0.0452
Ticker: MET, Coverage: 0.8852
Avg. length: 0.0563
Ticker: MDT, Coverage: 0.8784
Avg. length: 0.0451
Ticker: MDLZ, Coverage: 0.8890
Avg. length: 0.0385
Ticker: MCD, Coverage: 0.8769
Avg. length: 0.0347
Ticker: MA, Coverage: 0.8890
Avg. length: 0.0504
Ticker: LOW, Coverage: 0.8792
Avg. length: 0.0518
Ticker: LMT, Coverage: 0.8950
Avg. length: 0.0420
Ticker: LLY, Cov

In [10]:
# print Coverage for for each ticker ACI
print('Coverage for each ticker ACI:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_ACI[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_ACI[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_ACI[i]['upper_bound'] - ticker_results_ACI[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker ACI:
Ticker: AAPL, Coverage: 0.8958
Avg. length: 0.0607
Ticker: INTU, Coverage: 0.8897
Avg. length: 0.0691
Ticker: ORCL, Coverage: 0.8852
Avg. length: 0.0539
Ticker: NVDA, Coverage: 0.8890
Avg. length: 0.1057
Ticker: NKE, Coverage: 0.8875
Avg. length: 0.0598
Ticker: NFLX, Coverage: 0.9094
Avg. length: 0.0825
Ticker: NEE, Coverage: 0.8860
Avg. length: 0.0536
Ticker: MSFT, Coverage: 0.8958
Avg. length: 0.0576
Ticker: MS, Coverage: 0.8950
Avg. length: 0.0618
Ticker: MRK, Coverage: 0.8927
Avg. length: 0.0434
Ticker: MO, Coverage: 0.8995
Avg. length: 0.0406
Ticker: MMM, Coverage: 0.8897
Avg. length: 0.0487
Ticker: MET, Coverage: 0.8973
Avg. length: 0.0612
Ticker: MDT, Coverage: 0.8958
Avg. length: 0.0476
Ticker: MDLZ, Coverage: 0.8943
Avg. length: 0.0398
Ticker: MCD, Coverage: 0.8912
Avg. length: 0.0368
Ticker: MA, Coverage: 0.9033
Avg. length: 0.0548
Ticker: LOW, Coverage: 0.8965
Avg. length: 0.0539
Ticker: LMT, Coverage: 0.8973
Avg. length: 0.0435
Ticker: LLY, Cov

In [11]:
from TestingFramework import binomial_test, geometric_conformal_test, dynamic_binary_regression_test, diebold_mariano_test

In [12]:
#1. Test on standard CP
# Testing
p_values_binom_too_many = []
p_values_binom_too_few = []
p_values_binom_two_sided = []
p_values_geom_UC = []
p_values_geom_ind = []
p_values_geom_joint = []
p_values_dynamic_binary = []

for i, ticker in enumerate(ticker_list):
    p_under_binom, p_over_binom, p_two_sid_binom = binomial_test(ticker_results_standard_CP[i]['violation'], alpha=0.1)
    p_values_binom_too_many.append(p_under_binom)
    p_values_binom_too_few.append(p_over_binom)
    p_values_binom_two_sided.append(p_two_sid_binom)
    p_UC, p_ind, p_joint = geometric_conformal_test(ticker_results_standard_CP[i]['violation'], alpha=0.1)
    p_values_geom_UC.append(p_UC)
    p_values_geom_ind.append(p_ind)
    p_values_geom_joint.append(p_joint)
    p_dynamic_binary = dynamic_binary_regression_test(ticker_results_standard_CP[i]['y_true'], ticker_results_standard_CP[i]['violation'], ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary.append(p_dynamic_binary)
    


# Print p-values for standard CP
print('P-values for standard CP:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint[i]:.4f}')
    print(f'Dynamic binary regression test: {p_values_dynamic_binary[i]:.4f}')
    print('---')





P-values for standard CP:
Ticker: AAPL
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
---
Ticker: INTU
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
---
Ticker: ORCL
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
---
Ticker: NVDA
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
---
Ticker: NKE
Binomi

In [13]:
#count for each tests the number of p-values > 0.05 (Tests passed)
count_binom_too_many = sum(p > 0.05 for p in p_values_binom_too_many)
count_binom_too_few = sum(p > 0.05 for p in p_values_binom_too_few)
count_binom_two_sided = sum(p > 0.05 for p in p_values_binom_two_sided)
count_geom_UC = sum(p > 0.05 for p in p_values_geom_UC)
count_geom_ind = sum(p > 0.05 for p in p_values_geom_ind)
count_geom_joint = sum(p > 0.05 for p in p_values_geom_joint)
count_dynamic_binary = sum(p > 0.05 for p in p_values_dynamic_binary)
print(f'Number of p-values > 0.05 for binomial test too many: {count_binom_too_many}')
print(f'Number of p-values > 0.05 for binomial test too few: {count_binom_too_few}')
print(f'Number of p-values > 0.05 for binomial test two sided: {count_binom_two_sided}')
print(f'Number of p-values > 0.05 for geometric UC test: {count_geom_UC}')
print(f'Number of p-values > 0.05 for geometric independent test: {count_geom_ind}')
print(f'Number of p-values > 0.05 for geometric joint test: {count_geom_joint}')
print(f'Number of p-values > 0.05 for dynamic binary regression test: {count_dynamic_binary}')

Number of p-values > 0.05 for binomial test too many: 7
Number of p-values > 0.05 for binomial test too few: 94
Number of p-values > 0.05 for binomial test two sided: 6
Number of p-values > 0.05 for geometric UC test: 6
Number of p-values > 0.05 for geometric independent test: 94
Number of p-values > 0.05 for geometric joint test: 11
Number of p-values > 0.05 for dynamic binary regression test: 1


In [14]:
# 2. Test on CQR
# Testing
p_values_binom_too_many_CQR = []
p_values_binom_too_few_CQR = []
p_values_binom_two_sided_CQR = []
p_values_geom_UC_CQR = []
p_values_geom_ind_CQR = []
p_values_geom_joint_CQR = []
p_values_dynamic_binary_CQR = []

for i, ticker in enumerate(ticker_list):
    p_under_binom_CQR, p_over_binom_CQR, p_two_sid_binom_CQR = binomial_test(ticker_results_CQR[i]['violation'], alpha=0.1)
    p_values_binom_too_many_CQR.append(p_under_binom_CQR)
    p_values_binom_too_few_CQR.append(p_over_binom_CQR)
    p_values_binom_two_sided_CQR.append(p_two_sid_binom_CQR)
    p_UC_CQR, p_ind_CQR, p_joint_CQR = geometric_conformal_test(ticker_results_CQR[i]['violation'], alpha=0.1)
    p_values_geom_UC_CQR.append(p_UC_CQR)
    p_values_geom_ind_CQR.append(p_ind_CQR)
    p_values_geom_joint_CQR.append(p_joint_CQR)
    p_dynamic_binary_CQR = dynamic_binary_regression_test(ticker_results_CQR[i]['y_true'], ticker_results_CQR[i]['violation'], ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary_CQR.append(p_dynamic_binary_CQR)

# Print p-values for CQR
print('P-values for CQR:')  
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many_CQR[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few_CQR[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided_CQR[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC_CQR[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind_CQR[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint_CQR[i]:.4f}')
    print(f'Dynamic binary regression test: {p_values_dynamic_binary_CQR[i]:.4f}')
    print('---')

P-values for CQR:
Ticker: AAPL
Binomial test too many: 0.0152
Binomial test too few: 0.9878
Binomial test two sided: 0.0278
Geometric UC test: 0.0311
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0820
---
Ticker: INTU
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.9860
Dynamic binary regression test: 0.0007
---
Ticker: ORCL
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 1.0000
Geometric joint test: 0.0865
Dynamic binary regression test: 0.0000
---
Ticker: NVDA
Binomial test too many: 0.0002
Binomial test too few: 0.9999
Binomial test two sided: 0.0003
Geometric UC test: 0.0004
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0016
---
Ticker: NKE
Binomial test 

In [15]:
#count for each tests the number of p-values > 0.05 (Tests passed) for CQR	
count_binom_too_many_CQR = sum(p > 0.05 for p in p_values_binom_too_many_CQR)
count_binom_too_few_CQR = sum(p > 0.05 for p in p_values_binom_too_few_CQR)
count_binom_two_sided_CQR = sum(p > 0.05 for p in p_values_binom_two_sided_CQR)
count_geom_UC_CQR = sum(p > 0.05 for p in p_values_geom_UC_CQR)
count_geom_ind_CQR = sum(p > 0.05 for p in p_values_geom_ind_CQR)
count_geom_joint_CQR = sum(p > 0.05 for p in p_values_geom_joint_CQR)
count_dynamic_binary_CQR = sum(p > 0.05 for p in p_values_dynamic_binary_CQR)

print(f'Number of p-values > 0.05 for binomial test too many CQR: {count_binom_too_many_CQR}')
print(f'Number of p-values > 0.05 for binomial test too few CQR: {count_binom_too_few_CQR}')
print(f'Number of p-values > 0.05 for binomial test two sided CQR: {count_binom_two_sided_CQR}')
print(f'Number of p-values > 0.05 for geometric UC test CQR: {count_geom_UC_CQR}')
print(f'Number of p-values > 0.05 for geometric independent test CQR: {count_geom_ind_CQR}')
print(f'Number of p-values > 0.05 for geometric joint test CQR: {count_geom_joint_CQR}')
print(f'Number of p-values > 0.05 for dynamic binary regression test CQR: {count_dynamic_binary_CQR}')

Number of p-values > 0.05 for binomial test too many CQR: 20
Number of p-values > 0.05 for binomial test too few CQR: 95
Number of p-values > 0.05 for binomial test two sided CQR: 23
Number of p-values > 0.05 for geometric UC test CQR: 22
Number of p-values > 0.05 for geometric independent test CQR: 96
Number of p-values > 0.05 for geometric joint test CQR: 90
Number of p-values > 0.05 for dynamic binary regression test CQR: 31


In [16]:
# 3. Test on ACI
# Testing
p_values_binom_too_many_ACI = []
p_values_binom_too_few_ACI = []
p_values_binom_two_sided_ACI = []
p_values_geom_UC_ACI = []
p_values_geom_ind_ACI = []
p_values_geom_joint_ACI = []
p_values_dynamic_binary_ACI = []

for i, ticker in enumerate(ticker_list):
    p_under_binom_ACI, p_over_binom_ACI, p_two_sid_binom_ACI = binomial_test(ticker_results_ACI[i]['violation'], alpha=0.1)
    p_values_binom_too_many_ACI.append(p_under_binom_ACI)
    p_values_binom_too_few_ACI.append(p_over_binom_ACI)
    p_values_binom_two_sided_ACI.append(p_two_sid_binom_ACI)
    p_UC_ACI, p_ind_ACI, p_joint_ACI = geometric_conformal_test(ticker_results_ACI[i]['violation'], alpha=0.1)
    p_values_geom_UC_ACI.append(p_UC_ACI)
    p_values_geom_ind_ACI.append(p_ind_ACI)
    p_values_geom_joint_ACI.append(p_joint_ACI)
    p_dynamic_binary_ACI = dynamic_binary_regression_test(ticker_results_CQR[i]['y_true'], ticker_results_ACI[i]['violation'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary_ACI.append(p_dynamic_binary_ACI)

# Print p-values for ACI
print('P-values for ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many_ACI[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few_ACI[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided_ACI[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC_ACI[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind_ACI[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint_ACI[i]:.4f}')
    print(f'Dynamic binary regression test: {p_values_dynamic_binary_ACI[i]:.4f}')
    print('---')

P-values for ACI:
Ticker: AAPL
Binomial test too many: 0.3168
Binomial test too few: 0.7147
Binomial test two sided: 0.6142
Geometric UC test: 0.5831
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0000
---
Ticker: INTU
Binomial test too many: 0.1160
Binomial test too few: 0.9005
Binomial test two sided: 0.2160
Geometric UC test: 0.2433
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0000
---
Ticker: ORCL
Binomial test too many: 0.0421
Binomial test too few: 0.9652
Binomial test two sided: 0.0740
Geometric UC test: 0.0685
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0000
---
Ticker: NVDA
Binomial test too many: 0.0995
Binomial test too few: 0.9151
Binomial test two sided: 0.1839
Geometric UC test: 0.1711
Geometric independent test: 1.0000
Geometric joint test: 1.0000
Dynamic binary regression test: 0.0000
---
Ticker: NKE
Binomial test 

In [17]:
#count for each tests the number of p-values > 0.05 (Tests passed) for ACI
count_binom_too_many_ACI = sum(p > 0.05 for p in p_values_binom_too_many_ACI)
count_binom_too_few_ACI = sum(p > 0.05 for p in p_values_binom_too_few_ACI)
count_binom_two_sided_ACI = sum(p > 0.05 for p in p_values_binom_two_sided_ACI)
count_geom_UC_ACI = sum(p > 0.05 for p in p_values_geom_UC_ACI)
count_geom_ind_ACI = sum(p > 0.05 for p in p_values_geom_ind_ACI)
count_geom_joint_ACI = sum(p > 0.05 for p in p_values_geom_joint_ACI)
count_dynamic_binary_ACI = sum(p > 0.05 for p in p_values_dynamic_binary_ACI)
print(f'Number of p-values > 0.05 for binomial test too many: {count_binom_too_many_ACI}')
print(f'Number of p-values > 0.05 for binomial test too few: {count_binom_too_few_ACI}')
print(f'Number of p-values > 0.05 for binomial test two sided: {count_binom_two_sided_ACI}')
print(f'Number of p-values > 0.05 for geometric UC test: {count_geom_UC_ACI}')
print(f'Number of p-values > 0.05 for geometric independent test: {count_geom_ind_ACI}')
print(f'Number of p-values > 0.05 for geometric joint test: {count_geom_joint_ACI}')
print(f'Number of p-values > 0.05 for dynamic binary regression test: {count_dynamic_binary_ACI}')

Number of p-values > 0.05 for binomial test too many: 90
Number of p-values > 0.05 for binomial test too few: 96
Number of p-values > 0.05 for binomial test two sided: 94
Number of p-values > 0.05 for geometric UC test: 94
Number of p-values > 0.05 for geometric independent test: 96
Number of p-values > 0.05 for geometric joint test: 96
Number of p-values > 0.05 for dynamic binary regression test: 1


In [18]:
# 5. Diebold Marino Test between standard CP, CQR, ACI and adaptive_cqr

# Testing
p_values_diebold_mariano_std_CQR = []
DM_stat_std_CQR = []
p_values_diebold_mariano_std_ACI = []
DM_stat_std_ACI = []
p_values_diebold_mariano_CQR_ACI = []
DM_stat_CQR_ACI = []

for i, ticker in enumerate(ticker_list):
    DM_stat, p_value = diebold_mariano_test(ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], ticker_results_CQR[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_std_CQR.append(p_value)
    DM_stat_std_CQR.append(DM_stat)
    DM_stat, p_value = diebold_mariano_test(ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], ticker_results_ACI[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_std_ACI.append(p_value)
    DM_stat_std_ACI.append(DM_stat)
    DM_stat, p_value = diebold_mariano_test(ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], ticker_results_ACI[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_CQR_ACI.append(p_value)
    DM_stat_CQR_ACI.append(DM_stat)


# Print DM stat and p-values for standard CP and CQR
print('DM stat and p-values for standard CP and CQR:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_std_CQR[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_std_CQR[i]:.4f}')
    print('---')
print('#######################################################################################')
# Print DM stat and p-values for standard CP and ACI
print('DM stat and p-values for standard CP and ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_std_ACI[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_std_ACI[i]:.4f}')
    print('---')
print('#######################################################################################')
# Print DM stat and p-values for CQR and ACI
print('DM stat and p-values for CQR and ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_CQR_ACI[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_CQR_ACI[i]:.4f}')
    print('---')

Mean of d: 0.014358796030104868
DM variance: 5.781695671120415e-05
DM stat: 1.8883844355321864, p-value: 0.05897435931307671
Mean of d: 0.0067096596022275055
DM variance: 2.7368171206366727e-05
DM stat: 1.2825597274516534, p-value: 0.1996463670919928
Mean of d: -0.007649136427877365
DM variance: 2.9278827980573033e-05
DM stat: -1.4136293806316875, p-value: 0.15747075002784539
Mean of d: 0.03640352208701545
DM variance: 0.00013633046522396622
DM stat: 3.1177906181153836, p-value: 0.0018221220288119344
Mean of d: 0.030732221527785583
DM variance: 0.00011853785842163151
DM stat: 2.822704507020312, p-value: 0.0047620439886435495
Mean of d: -0.00567130055922987
DM variance: 3.003132853225607e-05
DM stat: -1.0348928619862416, p-value: 0.3007189485328947
Mean of d: 0.0161124630274819
DM variance: 7.127727820599295e-05
DM stat: 1.9084745817444702, p-value: 0.05632990536896432
Mean of d: 0.009823900836152021
DM variance: 3.061043218586273e-05
DM stat: 1.7756167748008518, p-value: 0.075796103208

In [19]:
# for each pair make 3 cat: method 1 better with p-value > 0.05, method 2 better with p-value, tie (uncertain) 
count_std_CQR_better = 0
count_std_CQR_worse = 0
count_std_CQR_tie = 0
count_std_ACI_better = 0
count_std_ACI_worse = 0
count_std_ACI_tie = 0
count_CQR_ACI_better = 0
count_CQR_ACI_worse = 0
count_CQR_ACI_tie = 0

for i, ticker in enumerate(ticker_list):
    if p_values_diebold_mariano_std_CQR[i] < 0.05:
        if DM_stat_std_CQR[i] < 0:
            count_std_CQR_better += 1
        elif DM_stat_std_CQR[i] > 0:
            count_std_CQR_worse += 1
    else:
        count_std_CQR_tie += 1
        print(f'Ticker: {ticker}')
        print(f'DM stat: {DM_stat_std_CQR[i]:.4f}')
        print(f'p-value: {p_values_diebold_mariano_std_CQR[i]:.4f}')
        print('---')
    if p_values_diebold_mariano_std_ACI[i] < 0.05:
        if DM_stat_std_ACI[i] < 0:
            count_std_ACI_better += 1
        elif DM_stat_std_ACI[i] > 0:
            count_std_ACI_worse += 1
    else:
        count_std_ACI_tie += 1
    if p_values_diebold_mariano_CQR_ACI[i] < 0.05:
        if DM_stat_CQR_ACI[i] < 0:
            count_CQR_ACI_better += 1
        elif DM_stat_CQR_ACI[i] > 0:
            count_CQR_ACI_worse += 1
    else:
        count_CQR_ACI_tie += 1

Ticker: AAPL
DM stat: 1.8884
p-value: 0.0590
---
Ticker: ORCL
DM stat: 1.9085
p-value: 0.0563
---
Ticker: NFLX
DM stat: 1.6342
p-value: 0.1022
---
Ticker: MSFT
DM stat: 1.6780
p-value: 0.0933
---
Ticker: MS
DM stat: 1.5930
p-value: 0.1112
---
Ticker: MRK
DM stat: 0.9220
p-value: 0.3565
---
Ticker: MO
DM stat: 1.4251
p-value: 0.1541
---
Ticker: MMM
DM stat: 1.6775
p-value: 0.0934
---
Ticker: MDT
DM stat: 1.6401
p-value: 0.1010
---
Ticker: MDLZ
DM stat: 0.9266
p-value: 0.3541
---
Ticker: MCD
DM stat: 1.5480
p-value: 0.1216
---
Ticker: LOW
DM stat: 1.6705
p-value: 0.0948
---
Ticker: LMT
DM stat: 1.6342
p-value: 0.1022
---
Ticker: LLY
DM stat: 1.5496
p-value: 0.1212
---
Ticker: LIN
DM stat: 1.9442
p-value: 0.0519
---
Ticker: KO
DM stat: 1.7473
p-value: 0.0806
---
Ticker: PEP
DM stat: 1.3334
p-value: 0.1824
---
Ticker: PFE
DM stat: 1.8640
p-value: 0.0623
---
Ticker: PG
DM stat: 1.4001
p-value: 0.1615
---
Ticker: WMT
DM stat: 1.5494
p-value: 0.1213
---
Ticker: VZ
DM stat: 1.2737
p-value: 0.2

In [20]:
print(f'Count standard CP better than CQR: {count_std_CQR_better}')
print(f'Count standard CP worse than CQR: {count_std_CQR_worse}')
print(f'Count standard CP tie with CQR: {count_std_CQR_tie}')
print('#######################################################################################')
print(f'Count standard CP better than ACI: {count_std_ACI_better}')
print(f'Count standard CP worse than ACI: {count_std_ACI_worse}')
print(f'Count standard CP tie with ACI: {count_std_ACI_tie}')
print('#######################################################################################')
print(f'Count CQR better than ACI: {count_CQR_ACI_better}')
print(f'Count CQR worse than ACI: {count_CQR_ACI_worse}')
print(f'Count CQR tie with ACI: {count_CQR_ACI_tie}')


Count standard CP better than CQR: 0
Count standard CP worse than CQR: 36
Count standard CP tie with CQR: 60
#######################################################################################
Count standard CP better than ACI: 0
Count standard CP worse than ACI: 21
Count standard CP tie with ACI: 75
#######################################################################################
Count CQR better than ACI: 0
Count CQR worse than ACI: 0
Count CQR tie with ACI: 96
